In [1]:
import numpy as np
from scipy import sparse
import itertools
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import pandas as pd
from resources import *
from scipy.stats import hmean
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from LightFM_Modeling import *
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import csr_matrix
from typing import Tuple, Dict, List
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split


e:\DS-projects\venv38-64\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## Data load

In [2]:
interactions_matrix = pd.read_csv("interactions.csv").values

# Получите количество пользователей и элементов
n_users, n_items = interactions_matrix.shape

# Преобразуйте матрицу взаимодействий в набор данных
data = []
for user_id in range(n_users):
    for item_id in range(n_items):
        interaction = interactions_matrix[user_id, item_id]
        data.append([user_id, item_id, interaction])
        
# Define parameters for each model
xgboost_params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "max_depth": 6,
    "learning_rate": 0.01,
}

lightgbm_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "max_depth": 6,
    "learning_rate": 0.01,
    "num_leaves": 20,
}

catboost_params = {
    "loss_function": "Logloss",
    "eval_metric": "Logloss",
    "depth": 6,
    "learning_rate": 0.01,
    "iterations": 100,
    "verbose": 0,
}

In [ ]:
data = np.array(data)
X = data[:, :2]  # признаки (user_id, item_id)
y = data[:, 2]   # целевая переменная (interaction)

# Разделите данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Разделите данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


interactions_df = pd.DataFrame(data, columns=["user_id", "item_id", "interaction"])
interactions_df.to_csv("full_interactions.csv", index=False)

In [ ]:

# Обучите и оцените модели градиентного бустинга с помощью представленных ранее функций
xgboost_model = train_xgboost(X_train, y_train, X_test, y_test, xgboost_params)
lightgbm_model = train_lightgbm(X_train, y_train, X_test, y_test, lightgbm_params)
catboost_model = train_catboost(X_train, y_train, X_test, y_test, catboost_params)

# Оцените модели
xgboost_eval = evaluate_classification(y_test, xgboost_model["test_predictions"])
lightgbm_eval = evaluate_classification(y_test, lightgbm_model["test_predictions"])
catboost_eval = evaluate_classification(y_test, catboost_model["test_predictions"])

print("XGBoost Evaluation:", xgboost_eval)
print("LightGBM Evaluation:", lightgbm_eval)
print("CatBoost Evaluation:", catboost_eval)

In [2]:
# Load and preprocess the data
interactions_csv_path = "interactions.csv"
X_reduced, X_2d, y = load_and_preprocess_data(interactions_csv_path)

e:\DS-projects\venv38-64\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(


In [3]:
# interactions_csv_path = "interactions.csv"
# X, X_2d = visualize_interactions_data_with_tsne(interactions_csv_path)


## K means

In [4]:
# # Run KMeans visualization on the 2D projected data
# elbow_data = run_kmeans_visualization(X, X_2d)


In [5]:
# # Plot the "elbow plot" to help determine the optimal value of k
# plt.plot(range(2, 11), elbow_data, "bo-")
# plt.xlabel("Number of Clusters (k)")
# plt.ylabel("Inertia")
# plt.title("Elbow Plot")
# plt.show()


## Gradient boosting

In [8]:

def evaluate_gradient_boosting_models(X: np.ndarray, y: np.ndarray):
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train and evaluate XGBoost Classifier
    xgb_results = train_xgboost(X_train, y_train, X_test, y_test, xgboost_params)
    xgb_evaluation = evaluate_classification(y_test, xgb_results["test_predictions"])
    print("XGBoost Classifier Evaluation:")
    print(xgb_evaluation)

    # Train and evaluate LightGBM Classifier
    lgbm_results = train_lightgbm(X_train, y_train, X_test, y_test, lightgbm_params)
    lgbm_evaluation = evaluate_classification(y_test, lgbm_results["test_predictions"])
    print("LightGBM Classifier Evaluation:")
    print(lgbm_evaluation)

    # Train and evaluate CatBoost Classifier
    cat_results = train_catboost(X_train, y_train, X_test, y_test, catboost_params)
    cat_evaluation = evaluate_classification(y_test, cat_results["test_predictions"])
    print("CatBoost Classifier Evaluation:")
    print(cat_evaluation)


In [9]:
evaluate_gradient_boosting_models(X_reduced, y)

XGBoost Classifier Evaluation:
{'accuracy': 1.0, 'f1': 1.0}
[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 20971, number of negative: 0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 20971, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Info] Start training from score 34.539576
[LightGBM] [Info] [binary:BoostFromScore]: pavg=1.000000 -> initscore=34.539576
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[1]	valid_0's binary_logloss: 0
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[2]	valid_0's binary_logloss: 0
[LightGBM] [Warning] Stopped training because ther

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/target_converter.cpp:375: Target contains only one unique value